In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
b = pd.read_csv('brfss2014_short.csv')

In [67]:
# Get rid of special codes like "did not answer" or "don't know"
# We may want to look into replacing these with the median value or something
# b = b[b.children != 88]
b = b[b.children != 89]
b = b[b.children != 99]
b = b[b.sleptim1 != 77]
b = b[b.sleptim1 != 99]
b = b[b.marital != 9]
b = b[b.employ1 != 9]
b = b[b.income2 != 77]
b = b[b.income2 != 99]
b = b[b.weight2 != 9999]
b = b[b.useequip != 7]
b = b[b.useequip != 9]
b = b[b['x.ageg5yr'] != 14]

In [4]:
b.head()

,Unnamed: 0,x.state,fmonth,idate,imonth,iday,iyear,dispcode,seqno,x.psu,...,x.fobtfs,x.crcrec,x.aidtst3,x.impeduc,x.impmrtl,x.imphome,rcsbrac1,rcsrace1,rchisla1,rcsbirth
0,1,1,1,1172014,1,17,2014,1100,2014000001,2014000001,...,2,1,2,5,1,1,NaN,NaN,NaN,NaN
1,2,1,1,1072014,1,7,2014,1100,2014000002,2014000002,...,2,2,2,4,1,1,NaN,NaN,NaN,NaN
2,3,1,1,1092014,1,9,2014,1100,2014000003,2014000003,...,2,2,2,6,1,1,NaN,NaN,NaN,NaN
3,4,1,1,1072014,1,7,2014,1100,2014000004,2014000004,...,2,1,2,6,3,1,NaN,NaN,NaN,NaN
4,5,1,1,1162014,1,16,2014,1100,2014000005,2014000005,...,2,1,2,5,1,1,NaN,NaN,NaN,NaN


In [100]:
print "Number of rows:", len(b)
print "Number of columns:", len(b.columns)

Number of rows: 7654
Number of columns: 280


These are the relevant rows that we'll be looking at:

In [71]:
sleptim = b['sleptim1']
marital = b['marital']
children = b['children']
employ = b['employ1']
income = b['income2']
weight = b['weight2']
height = b['height3']
sex = b['sex']
useequip = b['useequip']
age = b['x.ageg5yr']

In [72]:
age.value_counts()

9     946
10    902
8     878
7     825
11    651
6     575
5     494
13    467
12    431
4     413
3     388
2     352
1     332
Name: x.ageg5yr, dtype: int64

Demographic Average:

In [103]:
def getAgeCat(age):
    """
    Returns the age category number to use in the dataset given
    an actual integer age.
    """
    if age <= 24: return 1
    if age <= 29: return 2
    if age <= 34: return 3
    if age <= 39: return 4
    if age <= 44: return 5
    if age <= 49: return 6
    if age <= 54: return 7
    if age <= 59: return 8
    if age <= 64: return 9
    if age <= 69: return 10
    if age <= 74: return 11
    if age <= 79: return 12
    return 13

# Employment Code:
# 1: Employed for wages
# 2: Self-employed
# 3: Out of work for 1 year or more
# 4: Out of work for less than 1 year
# 5: A homemaker
# 6: A student
# 7: Retired
# 8: Unable to work
# 9: Refused

def getSleep(age, sex, married, children, employment):
    """
    Returns the average sleep that a person with given
    demographic parameters gets per night.
    age: int
    sex: int where 1 is male and 2 is female
    married: boolean whether they're married
    children: boolean whether they have children
    employment: int describing emplyment according to above code
    """
    df = b.copy()
    df = df[df.sex == sex]
    df = df[df['x.ageg5yr'] == getAgeCat(age)]
    if married:
        df = df[df.marital == 1]
    else:
        df = df[df.marital != 1]
    if children:
        df = df[df.children != 88]
    else:
        df = df[df.children == 88]
    df = df[df.employ1 == employment]
    print "number in this demographic:",len(df)
    return df['sleptim1'].mean()
getSleep(23,1, False, False, 1)

number in this demographic: 51


6.3137254901960782

Random Forest:

In [63]:
# predictors = ['marital', 'children', 'employ1', 'income2', 'weight2', 'height3', 'sex', 'useequip']
predictors = ['marital', 'children', 'employ1', 'sex', 'x.ageg5yr']

b = b.dropna(subset=predictors)

# Initialize our algorithm class
# alg = LinearRegression()
alg = RandomForestClassifier()
# Generate cross validation folds for the titanic dataset.  It return the row indices corresponding to train and test.
# We set random_state to ensure we get the same splits every time we run this.
# kf = KFold(b.shape[0], n_folds=3, random_state=1)
kf = KFold(b.shape[0], n_folds=3)

predictions = []
for train, test in kf:
    # The predictors we're using the train the algorithm.  Note how we only take the rows in the train folds.
    train_predictors = (b[predictors].iloc[train,:])
    # The target we're using to train the algorithm.
    train_target = b["sleptim1"].iloc[train]
    # Training the algorithm using the predictors and target.
    alg.fit(train_predictors, train_target)
    # We can now make predictions on the test fold
    test_predictions = alg.predict(b[predictors].iloc[test,:])
    predictions.append(test_predictions)
    
# The predictions are in three separate numpy arrays.  Concatenate them into one.  
# We concatenate them on axis 0, as they only have one axis.
predictions = np.concatenate(predictions, axis=0)

# Map predictions to outcomes (only possible outcomes are integers)
# predictions[predictions > .5] = 1
# predictions[predictions <=.5] = 0
# predictions[predictions > 1] = 8
# print predictions[:10]
# print b['sleptim1'][:10]



# print predictions[predictions == b["sleptim1"]][:10]

tot = 0
# for i in range(len(predictions)):
#     if predictions[i] == b['sleptim1'][i]:
#         tot += 1

for index, row in b.iterrows():
    if index < len(b):
        if predictions[index] == row['sleptim1']:
#         if 8 == row['sleptim1']:
            tot += 1
#     else:
#         print 'bad sleptim:',row['sleptim1']
print 'total accurate:',tot
print 'accuracy percent:', float(tot)/ len(predictions)  

# accuracy = sum(predictions[predictions == b["sleptim1"]]) / float(len(predictions))
# print accuracy

# Initialize our algorithm
# alg = LogisticRegression(random_state=1)
# alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)
alg = RandomForestClassifier()

# Compute the accuracy score for all the cross validation folds.  (much simpler than what we did before!)
scores = cross_validation.cross_val_score(alg, b[predictors], b["sleptim1"], cv=3)
# Take the mean of the scores (because we have one for each fold)
print scores.mean()

total accurate: 1381
accuracy percent: 0.179981754203
0.270799302283
